# Ray RLlib Multi-Armed Bandits - A Simple Bandit Example

© 2019-2021, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademyLogo.png)

Let's explore a very simple contextual bandit example with three arms. We'll run trials using RLlib and [Tune](http://tune.io), Ray's hyperparameter tuning library. 

In [1]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import random
import time
import ray

We define the bandit as a subclass of an OpenAI Gym environment. We set the action space to have three discrete variables, one action for each arm, and an observation space (the context) in the range -1.0 to 1.0, inclusive. (See the [configuring environments](https://docs.ray.io/en/latest/rllib-env.html#configuring-environments) documentation for more details about creating custom environments.)

There are two contexts defined. Note that we'll randomly pick one of them to use when `reset` is called, but it stays fixed (static) throughout the episode (the set of steps between calls to `reset`).

In [2]:
class SimpleContextualBandit (gym.Env):
    def __init__ (self, config=None):
        self.action_space = Discrete(3)     # 3 arms
        self.observation_space = Box(low=-1., high=1., shape=(2, ), dtype=np.float64)  # Random (x,y), where x,y from -1 to 1
        self.current_context = None
        self.rewards_for_context = {        # 2 contexts: -1 and 1
            -1.: [-10, 0, 10],
            1.: [10, 0, -10],
        }

    def reset (self):
        self.current_context = random.choice([-1., 1.])
        return np.array([-self.current_context, self.current_context])

    def step (self, action):
        reward = self.rewards_for_context[self.current_context][action]
        return (np.array([-self.current_context, self.current_context]), reward, True,
                {
                    "regret": 10 - reward
                })

    def __repr__(self):
        return f'SimpleContextualBandit(action_space={self.action_space}, observation_space={self.observation_space}, current_context={self.current_context}, rewards per context={self.rewards_for_context})'

Look at the definition of `self.rewards_for_context`. For context `-1.`, choosing the **third** arm (index 2 in the array) maximizes the reward, yielding `10.0` for each pull. Similarly, for context `1.`, choosing the **first** arm (index 0 in the array) maximizes the reward. It is never advantageous to choose the second arm.

We'll see if our training results agree ;)

Try repeating the next two code cells enough times to see the `current_context` set to `1.0` and `-1.0`, which is initialized randomly in `reset()`.

In [3]:
bandit = SimpleContextualBandit()
observation = bandit.reset()
f'Initial observation = {observation}, bandit = {repr(bandit)}'

'Initial observation = [ 1. -1.], bandit = SimpleContextualBandit(action_space=Discrete(3), observation_space=Box(-1.0, 1.0, (2,), float64), current_context=-1.0, rewards per context={-1.0: [-10, 0, 10], 1.0: [10, 0, -10]})'

The `bandit.current_context` and the observation of the current environment will remain fixed through the episode.

In [4]:
print(f'current_context = {bandit.current_context}')

for i in range(10):
    action = bandit.action_space.sample()
    observation, reward, done, info = bandit.step(action)
    print(f'observation = {observation}, action = {action}, reward = {reward:4d}, done = {str(done):5s}, info = {info}')

current_context = -1.0
observation = [ 1. -1.], action = 1, reward =    0, done = True , info = {'regret': 10}
observation = [ 1. -1.], action = 2, reward =   10, done = True , info = {'regret': 0}
observation = [ 1. -1.], action = 2, reward =   10, done = True , info = {'regret': 0}
observation = [ 1. -1.], action = 1, reward =    0, done = True , info = {'regret': 10}
observation = [ 1. -1.], action = 0, reward =  -10, done = True , info = {'regret': 20}
observation = [ 1. -1.], action = 2, reward =   10, done = True , info = {'regret': 0}
observation = [ 1. -1.], action = 2, reward =   10, done = True , info = {'regret': 0}
observation = [ 1. -1.], action = 2, reward =   10, done = True , info = {'regret': 0}
observation = [ 1. -1.], action = 0, reward =  -10, done = True , info = {'regret': 20}
observation = [ 1. -1.], action = 2, reward =   10, done = True , info = {'regret': 0}


Look at the `current_context`. If it's `1.0`, does the `0` (first) action yield the highest reward and lowest regret? If it's `-1.0`, does the `2` (third) action yield the highest reward and lowest regret? The `1` (second) action always returns `0` reward, so it's never optimal. 

## Using LinUCB

For this simple example, we can easily determine the best actions to take. Let's see how well our system does. We'll train with [LinUCB](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-upper-confidence-bound-contrib-linucb), a linear version of _Upper Confidence Bound_, for the exploration-exploitation strategy. _LinUCB_ assumes a linear dependency between the expected reward of an action and its context. Recall that a linear function is of the form $z = ax + by + c$, for example, where $x$, $y$, and $z$ are variables and $a$, $b$, and $c$ are constants. _LinUCB_ models the representation space using a set of linear predictors. Hence, the $Q_t(a)$ _value_ function discussed for UCB in the [previous lesson](02-Exploration-vs-Exploitation-Strategies.ipynb) is assumed to be a linear function here.

Look again at how we defined `rewards_for_context`. Is it linear as expected for _LinUCB_?

```python
self.rewards_for_context = {
    -1.: [-10, 0, 10],
    1.: [10, 0, -10],
}
```

Yes, for each arm, the reward is linear in the context. For example, the first arm has a reward of `-10` for context `-1.0` and `10` for context `1.0`. Crucially, the _same_ linear function that works for the first arm will work for the other two arms if you multiplied the constants in the linear function by `0` and `-1`, respectively. Hence, we expect _LinUCB_ to work well for this example.

Now use Tune to train the policy for this bandit. But first, we want to start Ray on your laptop or connect to the running Ray cluster if you are working on the Anyscale platform.

In [5]:
ray.init(ignore_reinit_error=True, num_cpus=2)

2021-05-22 22:14:20,187	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.138.0.19',
 'raylet_ip_address': '10.138.0.19',
 'redis_address': '10.138.0.19:6379',
 'object_store_address': '/tmp/ray/session_2021-05-22_22-14-19_694651_14969/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-22_22-14-19_694651_14969/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-05-22_22-14-19_694651_14969',
 'metrics_export_port': 33172,
 'node_id': '34757d3bf56c2f35980ec8e9f43e6cc3d691230bb75ba0a9abac276c'}

In [6]:
stop = {
    "training_iteration": 200,
    "timesteps_total": 100000,
    "episode_reward_mean": 10.0,
}

config = {
    "env": SimpleContextualBandit,
}

In [7]:
from ray.tune.progress_reporter import JupyterNotebookReporter

Calling `ray.tune.run` below would handle Ray initialization for us, if Ray isn't already running. If you want to prevent this and have Tune exit with an error when Ray isn't already initialized, then pass `ray_auto_init=False`.

In [8]:
analysis = ray.tune.run("contrib/LinUCB", config=config, stop=stop, 
    progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
    verbose=2,     # Change to 0 or 1 to reduce the output.
    )

Instructions for updating:
non-resource variables are not supported in the long term


Trial name,status,loc
contrib_LinUCB_SimpleContextualBandit_16e0a_00000,RUNNING,


(pid=15063) WARNING:tensorflow:From /home/ceteri/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=15063) Instructions for updating:
(pid=15063) non-resource variables are not supported in the long term
(pid=15063) 2021-05-22 22:14:37,110	INFO trainer.py:641 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=15063) 2021-05-22 22:14:37,266	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=15063) 2021-05-22 22:14:37,380	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


Trial contrib_LinUCB_SimpleContextualBandit_16e0a_00000 reported episode_reward_max=10.0,episode_reward_min=-10.0,episode_reward_mean=9.7,episode_len_mean=1.0,episodes_this_iter=100,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={'mean_env_wait_ms': 1.1072371265675762, 'mean_raw_obs_processing_ms': 0.33321002922435805, 'mean_inference_ms': 0.596402895332563, 'mean_action_processing_ms': 0.04105756778528196},off_policy_estimator={},num_healthy_workers=0,timers={'sample_time_ms': 1.024, 'sample_throughput': 976.224, 'learn_time_ms': 0.299, 'learn_throughput': 3344.74},info={'learner': {'default_policy': {'cumulative_regret': 30, 'update_latency': 0.00022101402282714844}}, 'num_steps_sampled': 100, 'num_steps_trained': 100},perf={'cpu_util_percent': 23.9, 'ram_util_percent': 32.8} with parameters={'env': <class '__main__.SimpleContextualBandit'>}.
Trial contrib_LinUCB_SimpleContextualBandit_16e0a_00000 reported episode_reward_max=10.0,episod

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
contrib_LinUCB_SimpleContextualBandit_16e0a_00000,TERMINATED,,2,0.399657,200,10,10,10,1


2021-05-22 22:14:37,975	INFO tune.py:450 -- Total run time: 5.82 seconds (3.08 seconds for the tuning loop).


(A lot of output is printed with `verbose` set to `2`. Use `0` for no output and `1` for short summaries.)

How long did it take?

In [9]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

   3.08 seconds,    0.05 minutes


We can see some of the final data as a dataframe:

In [10]:
df = analysis.dataframe(metric="episode_reward_mean", mode="max")
df

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,done,episodes_total,training_iteration,...,timers/learn_time_ms,timers/learn_throughput,info/num_steps_sampled,info/num_steps_trained,perf/cpu_util_percent,perf/ram_util_percent,info/learner/default_policy/cumulative_regret,info/learner/default_policy/update_latency,config/env,logdir
0,10.0,10.0,10.0,1.0,100,0,200,True,200,2,...,0.272,3670.521,200,200,NaN,NaN,30,0.000199,<class '__main__.SimpleContextualBandit'>,/home/ceteri/ray_results/contrib/LinUCB/contri...


The easiest way to inspect the progression of training is to use TensorBoard.

1. If you are runnng on the Anyscale Platform, click the _TensorBoard_ link. 
2. If you running this notebook on a laptop, open a terminal window using the `+` under the _Edit_ menu, run the following command, then open the URL shown.

```
tensorboard --logdir ~/ray_results 
```

You may have many data sets plotted from previous tutorial lessons. In the _Runs_ on the left, look for one named something like this:

```
contrib/LinUCB/contrib_LinUCB_SimpleContextualBandit_0_YYYY-MM-DD_HH-MM-SSxxxxxxxx  
```

If you have several of them, you want the one with the latest timestamp. To select just that one, click _toggler all runs_ below the list of runs, then select the one you want. You should see something like [this image](../../images/rllib/TensorBoard1.png).

The graph for the metric we were optimizing, the mean reward, is shown with a rectangle surrounding it. It improved steadily during the training runs. For this simple example, the reward mean is easily found in 200 steps.

## Exercise 1

Change the the `step` method to randomly change the `current_context` on each invocation:

```python
def step(self, action):
    result = super().step(action)
    self.current_context = random.choice([-1.,1.])
    return (np.array([-self.current_context, self.current_context]), reward, True,
            {
                "regret": 10 - reward
            })
```

Repeat the training and analysis. Does the training behavior change in any appreciable way? Why or why not?

See the [solutions notebook](solutions/Multi-Armed-Bandits-Solutions.ipynb) for discussion of this and the following exercises.

## Exercise 2

Recall the `rewards_for_context` we used:

```python
self.rewards_for_context = {
    -1.: [-10, 0, 10],
    1.: [10, 0, -10],
}
```

We said that Linear Upper Confidence Bound assumes a linear dependency between the expected reward of an action and its context. It models the representation space using a set of linear predictors.

Change the values for the rewards as follows, so they no longer have the same simple linear relationship:

```python
self.rewards_for_context = {
    -1.: [-10, 10, 0],
    1.: [0, 10, -10],
}
```

Run the training again and look at the results for the reward mean in TensorBoard. How successful was the training? How smooth is the plot for `episode_reward_mean`? How many steps were taken in the training?

## Exercise 3 (Optional)

We briefly discussed another algorithm for selecting the next action, _Thompson Sampling_, in the [previous lesson](02-Exploration-vs-Exploitation-Strategies.ipynb). Repeat exercises 1 and 2 using linear version, called _Linear Thompson Sampling_ ([RLlib documentation](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-thompson-sampling-contrib-lints)). To make this change, look at this code we used above:

```python
analysis = ray.tune.run("contrib/LinUCB", config=config, stop=stop, 
    progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
    verbose=1)
```

Change `contrib/LinUCB` to `contrib/LinTS`.  

We'll continue exploring usage of _LinUCB_ in the next lesson, [04 Linear Upper Confidence Bound](04-Linear-Upper-Confidence-Bound.ipynb) and _LinTS_ in the following lesson, [05 Thompson Sampling](05-Linear-Thompson-Sampling.ipynb).

In [11]:
ray.shutdown()